In [ ]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# Acessa as respostas aos recursos administrativos em pedidos da LAI no Estado de São Paulo
# Faz: o download dos PDFs, 
# transforma os arquivos em imagem, 
# faz uma leitura OCR, 
# captura o texto 
# e filtra por resposta deferidas e indeferidas
# Cria dataframe e arquivos com os resultados
# A rotina de leitura OCR foi adaptada de:
# https://www.geeksforgeeks.org/python-reading-contents-of-pdf-using-ocr-optical-character-recognition/
#

In [13]:
import requests
from urllib.request import urlopen
import wget
from bs4 import BeautifulSoup
import pandas as pd
import os
import time

In [14]:
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 

In [3]:
def captura_texto(arquivo):
    PDF_file = arquivo

    # Armazena todas as páginas do PDF em uma variável
    pages = convert_from_path(PDF_file, 500) 

    # Contador para armazenar imagens de cada página do PDF na imagem
    image_counter = 1

    # Itera todas as páginas armazenadas acima
    for page in pages: 
        # Declarando o nome do arquivo para cada página do PDF como JPG
        # Para cada página, o nome do arquivo será:
        # PDF page 1 -> page_1.jpg 
        # PDF page 2 -> page_2.jpg 
        # PDF page 3 -> page_3.jpg 
        # .... 
        # PDF page n -> page_n.jpg 
        filename = "page_"+str(image_counter)+".jpg"
      
        # Salve a imagem da página no sistema
        page.save(filename, 'JPEG') 
  
        # Incremente o contador para atualizar o nome do arquivo
        image_counter = image_counter + 1

    ''' 
    Parte 2 - Reconhecendo o texto das imagens usando o OCR
    '''
    
    # Variável para obter a contagem do número total de páginas
    filelimit = image_counter-1
  
    # Criando um arquivo de texto para gravar a saída
    outfile = "out_text.txt"
  
    # Abra o arquivo no modo de acréscimo para que
    # todo o conteúdo de todas as imagens seja adicionado ao mesmo arquivo
    f = open(outfile, "a") 
  
    # Iterar de 1 para o número total de páginas

    for i in range(1, filelimit + 1): 
        # Defina o nome do arquivo para reconhecer o texto
        # Novamente, esses arquivos serão:
        # page_1.jpg 
        # page_2.jpg 
        # .... 
        # page_n.jpg 
        filename = "page_"+str(i)+".jpg"
          
        # Reconhecer o texto como string na imagem usando pytesserct
        text = str(((pytesseract.image_to_string(Image.open(filename))))) 
  
        # O texto reconhecido é armazenado em uma variável de texto
        # Qualquer processamento de string pode ser aplicado ao texto
        # Aqui, a formatação básica foi feita:
        # Em muitos PDFs, no final da linha, se uma palavra não puder
        # ser totalmente escrita, um 'hífen' é adicionado.
        # O restante da palavra está escrito na próxima linha
        # Por exemplo: Este é um exemplo de texto desta palavra aqui GeeksF-
        # orGeeks está na metade da primeira linha, permanecendo na próxima.
        # Para remover isso, substituímos todos os '-\n' por ''.  
        text = text.replace('-\n', '')     
  
        # Por fim, escreve o texto processado no arquivo.
        f.write(text) 
  
    # Feche o arquivo depois de escrever todo o texto
    f.close() 
    
    # Abre o arquivo criado
    f=open("out_text.txt", "r")
    
    if f.mode == 'r':
        # Armazena o conteudo
        contents =f.read()
        conteudo = contents
    else:
        conteudo = ""
    
    # Retorna o conteudo para iteração principal
    return conteudo

In [ ]:
# URL onde estão os links dos PDFs

In [4]:
pagina = urlopen("http://www.ouvidoriageral.sp.gov.br/decisoesLAI.html")

In [ ]:
# Procura o local com os endereços dos arquivos

In [5]:
sopa =  BeautifulSoup(pagina, "lxml")

In [6]:
pdfs = sopa.findAll("table", {"class":"borderTD"})

In [7]:
lista = []
fixo = "http://www.ouvidoriageral.sp.gov.br/"

In [ ]:
# Faz um iteração na tabela de links e armazena os endereços

In [8]:
for table_ele in pdfs:
    
    for row in table_ele.findAll('tr'):
        
        cols = row.findAll('td')
        
        for item in cols:
            try:    
                url = item.find('a').get('href')
                site = fixo + url
                ano = url[9:13]
                numero = item.text
                dicionario = {"site": str(site).strip(),
                              "numero": str(numero).strip(),
                              "ano": str(ano).strip()
                             }
                lista.append(dicionario)
            except:
                None

In [ ]:
# Cria um dataframe

In [9]:
df_pdfs = pd.DataFrame(lista)

In [10]:
df_pdfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1261 entries, 0 to 1260
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   site    1261 non-null   object
 1   numero  1261 non-null   object
 2   ano     1261 non-null   object
dtypes: object(3)
memory usage: 29.7+ KB


In [11]:
df_pdfs.reset_index().tail()

,index,site,numero,ano
1256,1256,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,364,2016
1257,1257,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,365,2016
1258,1258,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,366,2016
1259,1259,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,368,2016
1260,1260,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,369,2016


In [15]:
# Seta o diretório de trabalho 

In [23]:
print (os.getcwd())

/home/abraji/Documentos/Code/lai_sp/repo


In [24]:
os.chdir("/home/abraji/Documentos/Code/lai_sp/repo/pdfs")

In [25]:
print (os.getcwd())

/home/abraji/Documentos/Code/lai_sp/repo/pdfs


In [26]:
dirname = "/home/abraji/Documentos/Code/lai_sp/repo/pdfs"

In [ ]:
# Cria uma lista vazia para os resultados

In [16]:
lista_final = []

In [ ]:
# Inicia iteração no dataframe de links de PDFs

In [ ]:
for num, row in df_pdfs.iterrows():
    # Cria variáveis com os itens do PDFs
    link = row['site']
    numero = row['numero']
    ano = row['ano']
    # O nome do arquivo fica nessa posição, mas pode ter um caracter indevido
    arquivo = link[50:]
    arquivo = arquivo.replace("/", "")
    
    print(arquivo)
    print(link)
    #time.sleep(2)
    
    try:
        # Faz o download do PDF
        wget.download(link)
        
        # Faz a rotina OCR, baseado no nome do arquivo na função acima
        texto = captura_texto(arquivo)
        
        # Cria um dicionário com dados do PDFs e seu conteúdo retornado
        dicionario = {"site": str(link).strip(),
                      "numero": str(numero).strip(),
                      "ano": str(ano).strip(),
                      "texto_do_documento": str(texto).strip()
                             }
        # Armazena na lista criada
        lista_final.append(dicionario)
        
        # Apaga o arquivo de conteudo do PDF atual
        os.remove("out_text.txt")
        # Apaga as imagens OCR do PDF atual
        test = os.listdir(dirname)
        for item in test:
            if item.endswith(".jpg"):
                os.remove(os.path.join(dirname, item))
        
    except:
        print("ERRRO")
        pass

In [ ]:
# Cria dataframe a partir da lista de conteudos de PDFs

In [18]:
df_pdfs_final = pd.DataFrame(lista_final)

In [19]:
df_pdfs_final.reset_index().head()

,index,site,numero,ano,texto_do_documento
0,0,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,004,2019,GOVERNO DO ESTADO DE SAO PAULO\nOUVIDORIA GERA...
1,1,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,005,2019,GOVERNO DO ESTADO DE SAO PAULO\nOUVIDORIA GERA...
2,2,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,006,2019,—\n\nGOVERNO DO ESTADO DE SAO PAULO\nOUVIDORIA...
3,3,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,007,2019,GOVERNO DO ESTADO DE SAO PAULO\nOUVIDORIA GERA...
4,4,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,008,2019,GOVERNO DO ESTADO DE SAO PAULO\nOUVIDORIA GERA...


In [27]:
df_pdfs_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   site                1253 non-null   object
 1   numero              1253 non-null   object
 2   ano                 1253 non-null   object
 3   texto_do_documento  1253 non-null   object
 4   texto_minusculo     1253 non-null   object
dtypes: object(5)
memory usage: 49.1+ KB


In [21]:
df_pdfs_final.to_csv("../laisp_textos.csv", index=False)

In [ ]:
# Carrega arquivo salvo se não quiser repetir o download OCR

In [5]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
df_pdfs_final = pd.read_csv("laisp_textos.csv", **kwargs)

In [ ]:
# Cria um campo em separado copia para fazer buscas

In [22]:
df_pdfs_final['texto_minusculo'] = df_pdfs_final['texto_do_documento']
df_pdfs_final['texto_minusculo'] = df_pdfs_final['texto_minusculo'].str.lower()

In [ ]:
# Procura termos relacionados a indeferimento

In [23]:
search_list = ["indeferido", "indeferimento", "negado provimento", "recurso nao conhecido", "recurso não conhecido", "perda de objeto"]
mask = df_pdfs_final['texto_minusculo'].str.contains('|'.join(search_list))
indeferido = df_pdfs_final[mask]
indeferido.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 519 entries, 0 to 1251
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   site                519 non-null    object
 1   numero              519 non-null    object
 2   ano                 519 non-null    object
 3   texto_do_documento  519 non-null    object
 4   texto_minusculo     519 non-null    object
dtypes: object(5)
memory usage: 24.3+ KB


In [24]:
indeferido.to_csv("indeferidos.csv", index=False)

In [ ]:
# Procura termos relacionados a deferimento

In [25]:
search_list = ["deferido", "deferimento", "provimento recursal"]
mask = df_pdfs_final['texto_minusculo'].str.contains('|'.join(search_list))
deferido = df_pdfs_final[mask]
deferido.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325 entries, 2 to 1251
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   site                325 non-null    object
 1   numero              325 non-null    object
 2   ano                 325 non-null    object
 3   texto_do_documento  325 non-null    object
 4   texto_minusculo     325 non-null    object
dtypes: object(5)
memory usage: 15.2+ KB


In [26]:
deferido.to_csv("deferidos.csv", index=False)